# Volcado Tabular de Vencimientos Explained con Otros Campos

In [36]:
from datetime import date
import pandas as pd
import requests

## Rutas

In [37]:
rutas = {
    "explained": "https://ms-tesoreria-analytics-staging.interconecta2.cl/v1/settlements/swaps/explained",
    "headers": "https://ms-tesoreria-analytics-staging.interconecta2.cl/operations/all",
}

## SDK

Estas son funciones que utilizan los endpoints adecuados para obtener la data necesaria para el volcado.

In [38]:
def get_explained(fecha: date, ruta: str):
    result = requests.get(f"{ruta}/{fecha}")
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

In [39]:
def get_headers(ruta: str):
    result = requests.get(ruta)
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

## Procedimiento

Se define la fecha de proceso.

In [40]:
fecha_proceso = date(2025, 7, 4)
f_proc = fecha_proceso.isoformat().replace("-", "")

Se obtienen los explained a fecha de proceso.

In [41]:
explained = get_explained(fecha_proceso, rutas["explained"])

Se obtienen los encabezados de la cartera vigente.

In [42]:
headers = get_headers(rutas["headers"])

Vemos un registro de los explained.

In [43]:
explained[0]

{'dealNumber': '2256',
 'legNumber': 1,
 'tipoPata': 'OVERNIGHT_INDEX',
 'recPay': 'A',
 'fechaInicial': '2025-01-06',
 'fechaFinal': '2025-07-04',
 'fechaInicialIndice': '2025-01-06',
 'fechaFinalIndice': '2025-07-04',
 'fechaPago': '2025-07-04',
 'nocional': 3000000000.0,
 'amortizacion': 0.0,
 'amortEsFlujo': True,
 'monedaNocional': 'CLP',
 'nombreIndice': 'ICPCLP',
 'valorIndiceInicial': 24801.68,
 'valorIndiceFinal': 25425.95,
 'valorTasa': 0.0506,
 'tipoTasa': 'LinAct360',
 'interes': 75478333.0,
 'flujo': 75478333.0,
 'spread': 0.0,
 'gearing': 1.0,
 'numDecimalesTasaEquivalente': 4,
 'fraccionAgno': '(fechaFinal - fechaInicial) / 360',
 'formulaValorTasaEquivalente': 'round((valorIndiceFinal / valorIndiceInicial - 1) * 1 / fraccionAgno, numDecimalesTasaEquivalente)',
 'formulaInteres': 'nocional * (formulaValorTasaEquivalente + spread) * fraccionAgno'}

Se observa que no tiene los campos de portfolio (gestión o contable) ni la estrategia contable. Se los agregamos con la data de los headers.

In [44]:
df_explained = pd.DataFrame.from_dict(explained)

Vemos la estructura de un header.

In [45]:
headers["5000"]

{'trade_date': {'fecha': '2022-11-29'},
 'deal_number': '5000',
 'counterparty': {'name': 'BANCO SANTANDER-SANTIAGO',
  'rut': {'rut': 97036000, 'dv': 'K'},
  'other': None},
 'portfolio': 'BANCA',
 'hedge_accounting': 'COB_TASA_INT_ACTIVOS',
 'product': 'SWAP_MONE',
 'currency_pair': 'CLFCLP',
 'price': None,
 'settlement_mechanism': 'C',
 'other': {'regulatory_portfolio': 'COBERTURA_VALOR_RAZONABLE'}}

In [46]:
def generate_new_columns(row):
    # Notar que headers se toma por closure
    operacion = headers.get(row.dealNumber)
    if operacion is None:
        operacion = {"dummy": 1}
    if operacion.get("other") is None:
        regulatory_portfolio = None
    else:
        regulatory_portfolio = operacion.get("other").get("regulatory_portfolio")
    return pd.Series({
        'portfolio': operacion.get("portfolio"),
        'hedgeAccounting': operacion.get("hedge_accounting"),
        'product': operacion.get("product"),
        'settlementMechanism': operacion.get("settlement_mechanism"),
        'regulatoryPortfolio': regulatory_portfolio,
    })

In [47]:
df_explained_ok = df_explained.join(df_explained.apply(
    generate_new_columns,
    axis=1,
))
del df_explained_ok["fraccionAgno"]
del df_explained_ok["formulaValorTasaEquivalente"]
del df_explained_ok["formulaInteres"]
del df_explained_ok["formulaInteresMonedaPago"]
del df_explained_ok["formulaAmortizacionMonedaPago"]

KeyError: 'formulaInteresMonedaPago'

In [ ]:
df_explained_ok

,dealNumber,legNumber,tipoPata,recPay,fechaInicial,fechaFinal,fechaInicialIndice,fechaFinalIndice,fechaPago,nocional,...,amortizacionMonedaPago,flujoMonedaPago,numDecimalesTasaEquivalente,codigoIndiceTasa,fxRateIndex,portfolio,hedgeAccounting,product,settlementMechanism,regulatoryPortfolio
0,4218,1,OVERNIGHT_INDEX_MCCY,A,2025-01-03,2025-07-03,2025-01-03,2025-07-03,2025-07-03,3.280000e+09,...,0.0,90104.730000,6.0,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
1,4218,2,FIXED_RATE_MCCY,P,2025-01-03,2025-07-03,NaN,NaN,2025-07-03,-3.280000e+09,...,0.0,-96371.076241,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
2,5643,1,FIXED_RATE,A,2025-01-03,2025-07-03,NaN,NaN,2025-07-03,2.812250e+09,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
3,5643,2,FIXED_RATE_MCCY,P,2025-01-03,2025-07-03,NaN,NaN,2025-07-03,-3.500000e+06,...,0.0,0.000000,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
4,5643,3,OVERNIGHT_INDEX,A,2025-01-03,2025-07-03,2025-01-03,2025-07-03,2025-07-08,2.812250e+09,...,NaN,NaN,4.0,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
5,5643,4,COMPOUNDED_OVERNIGHT_RATE_MCCY,P,2025-01-03,2025-07-03,NaN,NaN,2025-07-08,-3.500000e+06,...,0.0,-0.000000,8.0,SOFRRATE,USDOBS,SWAP,NO,SWAP_MONE,C,NEGOCIACION
6,5645,1,FIXED_RATE_MCCY,A,2025-01-03,2025-07-03,NaN,NaN,2025-07-03,2.000000e+06,...,0.0,0.000000,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
7,5645,2,FIXED_RATE,P,2025-01-03,2025-07-03,NaN,NaN,2025-07-03,-1.607000e+09,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
8,5645,3,COMPOUNDED_OVERNIGHT_RATE_MCCY,A,2025-01-03,2025-07-03,NaN,NaN,2025-07-08,2.000000e+06,...,0.0,0.000000,8.0,SOFRRATE,USDOBS,SWAP,NO,SWAP_MONE,C,NEGOCIACION
9,5645,4,OVERNIGHT_INDEX,P,2025-01-03,2025-07-03,2025-01-03,2025-07-03,2025-07-08,-1.607000e+09,...,NaN,NaN,4.0,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION


In [14]:
#df_explained_ok.to_csv(f"./{f_proc}_explained_plus_plus.csv")

Ahora vamos a convertir la dataframe en una vista por operación.

In [15]:
import numpy as np

Crear una función helper para agregar los datos correctamente

In [16]:
def aggregate_deal_data(group):
    """
    Aggregate deal data from leg-level to deal-level
    """
    # Get deal-level attributes (should be the same for all legs)
    deal_attrs = {
        'DealNumber': group['dealNumber'].iloc[0],
        'product': group['product'].iloc[0],
        'fechaFinal': group['fechaFinal'].iloc[0],
        'settlementMechanism': group['settlementMechanism'].iloc[0],
        'hedgeAccounting': group['hedgeAccounting'].iloc[0],
        'regulatoryPortfolio': group['regulatoryPortfolio'].iloc[0]
    }
    
    # Separate active (A) and passive (P) legs
    active_legs = group[group['recPay'] == 'A']
    passive_legs = group[group['recPay'] == 'P']
    
    # Process active legs
    if len(active_legs) > 0:
        deal_attrs['pataActivaAmortizacion'] = active_legs['amortizacion'].sum()
        deal_attrs['pataActivaMonedaNocional'] = active_legs['monedaNocional'].iloc[0]  # Assuming same currency for all active legs
        # Get fechaPago from lowest legNumber for active legs
        deal_attrs['pataActivaFechaPagoAmortizacion'] = active_legs.loc[active_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        active_flujo_values = active_legs['flujoMonedaPago'].fillna(active_legs['flujo'])
        deal_attrs['pataActivaFlujo'] = active_flujo_values.sum() if not active_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataActivaAmortizacion'] = np.nan
        deal_attrs['pataActivaMonedaNocional'] = np.nan
        deal_attrs['pataActivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataActivaFlujo'] = np.nan
    
    # Process passive legs
    if len(passive_legs) > 0:
        deal_attrs['pataPasivaAmortizacion'] = passive_legs['amortizacion'].sum()
        deal_attrs['pataPasivaMonedaNocional'] = passive_legs['monedaNocional'].iloc[0]  # Assuming same currency for all passive legs
        # Get fechaPago from lowest legNumber for passive legs
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = passive_legs.loc[passive_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        passive_flujo_values = passive_legs['flujoMonedaPago'].fillna(passive_legs['flujo'])
        deal_attrs['pataPasivaFlujo'] = passive_flujo_values.sum() if not passive_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataPasivaAmortizacion'] = np.nan
        deal_attrs['pataPasivaMonedaNocional'] = np.nan
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataPasivaFlujo'] = np.nan
    
    return pd.Series(deal_attrs)

Agrupamos por deal number

In [17]:
df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)

C:\Users\bencl\AppData\Local\Temp\ipykernel_56704\4184180563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)


Aplicar conversiones de nombres de producto

In [18]:
# Apply product name conversions
product_mapping = {
    'SWAP_ICP': 'Swap Promedio Cámara'.encode('utf-8').decode('utf-8'),
    'SWAP_MONE': 'Swap Moneda',
    'SWAP_TASA': 'Swap Tasas'
}
df_deal_level['product'] = df_deal_level['product'].map(product_mapping).fillna(df_deal_level['product'])

Y calcular monto neto a compensar en caso de modalidad compensación

In [19]:
df_deal_level['netoACompensar'] = np.nan
mask_compensation = df_deal_level['settlementMechanism'] == 'C'

df_deal_level.loc[mask_compensation, 'netoACompensar'] = (
    df_deal_level.loc[mask_compensation, 'pataActivaFlujo'].fillna(0) + 
    df_deal_level.loc[mask_compensation, 'pataPasivaFlujo'].fillna(0)
)

Y reordenamos las columnas

In [20]:
# Reorder columns to include the new netoACompensar column
column_order = [
    'DealNumber', 'product', 'fechaFinal', 'settlementMechanism', 'hedgeAccounting', 
    'regulatoryPortfolio', 'pataActivaAmortizacion', 'pataActivaMonedaNocional', 
    'pataActivaFechaPagoAmortizacion', 'pataActivaFlujo', 'pataPasivaAmortizacion',
    'pataPasivaMonedaNocional', 'pataPasivaFechaPagoAmortizacion', 'pataPasivaFlujo',
    'netoACompensar'
]

df_deal_level = df_deal_level[column_order]

Y vemos los resultados:

In [ ]:
print("Deal-level dataframe shape:", df_deal_level.shape)
print("\nFirst few rows:")
print(df_deal_level.head())

print("\nDataframe info:")
print(df_deal_level.info())

print("\nSample of deal-level data:")
print(df_deal_level.head(10))

Deal-level dataframe shape: (6, 15)

First few rows:
  DealNumber               product  fechaFinal settlementMechanism  \
0       4218  Swap Promedio Cámara  2025-07-03                   C   
1       5643           Swap Moneda  2025-07-03                   C   
2       5645           Swap Moneda  2025-07-03                   C   
3       5648           Swap Moneda  2025-07-03                   C   
4       6204  Swap Promedio Cámara  2025-07-03                   C   

  hedgeAccounting regulatoryPortfolio  pataActivaAmortizacion  \
0              NO         NEGOCIACION                     0.0   
1              NO         NEGOCIACION                     0.0   
2              NO         NEGOCIACION                     0.0   
3              NO         NEGOCIACION                     0.0   
4              NO         NEGOCIACION                     0.0   

  pataActivaMonedaNocional pataActivaFechaPagoAmortizacion  pataActivaFlujo  \
0                      CLP                      2025-07-

In [22]:
df_deal_level

,DealNumber,product,fechaFinal,settlementMechanism,hedgeAccounting,regulatoryPortfolio,pataActivaAmortizacion,pataActivaMonedaNocional,pataActivaFechaPagoAmortizacion,pataActivaFlujo,pataPasivaAmortizacion,pataPasivaMonedaNocional,pataPasivaFechaPagoAmortizacion,pataPasivaFlujo,netoACompensar
0,4218,Swap Promedio Cámara,2025-07-03,C,NO,NEGOCIACION,0.0,CLP,2025-07-03,9.010473e+04,0.0,CLP,2025-07-03,-9.637108e+04,-6.266346e+03
1,5643,Swap Moneda,2025-07-03,C,NO,NEGOCIACION,0.0,CLP,2025-07-03,6.178904e+07,0.0,USD,2025-07-03,0.000000e+00,6.178904e+07
2,5645,Swap Moneda,2025-07-03,C,NO,NEGOCIACION,0.0,USD,2025-07-03,0.000000e+00,0.0,CLP,2025-07-03,-3.692395e+07,-3.692395e+07
3,5648,Swap Moneda,2025-07-03,C,NO,NEGOCIACION,0.0,CLP,2025-07-03,1.180334e+08,0.0,USD,2025-07-03,0.000000e+00,1.180334e+08
4,6204,Swap Promedio Cámara,2025-07-03,C,NO,NEGOCIACION,0.0,CLP,2025-07-03,2.365972e+08,0.0,CLP,2025-07-03,-2.398703e+08,-3.273083e+06
5,6205,Swap Promedio Cámara,2025-07-03,C,NO,NEGOCIACION,0.0,CLP,2025-07-03,2.805207e+05,0.0,CLP,2025-07-03,-2.747095e+05,5.811172e+03


In [23]:
#df_deal_level.to_csv(f"./{f_proc}_deal_level.csv", index=False, encoding='utf-8')
#df_deal_level.to_excel(f"./{f_proc}_deal_level.xlsx", index=False)
df_deal_level.to_excel(rf"C:\Users\bencl\OneDrive - palace.cl\Documents\Palace\Banco Internacional\Treasury PM\06 Pruebas\{f_proc}_deal_level.xlsx", index=False)
